------------
# Generate Contour Data Files


## Initialize the Arrays and associated dataFrame objects
mARange and epsilonRange specify the range of parameter values we want to scan over
https://docs.scipy.org/doc/numpy/reference/generated/numpy.logspace.htmlm

In [ ]:
#########################################
# Define parameter space Ranges
#########################################
# Linear Spacing
# mALinRange = np.linspace(0.01, 10.0, 20)
# epsilonLinRange = np.linspace(10**-10, 10**-5, 20)

# Logarithmic Spacing
mALogRange = np.logspace(-2, 1, 100, base = 10)
epsilonLogRange = np.logspace(-10, -5, 100, base = 10)


#########################################
# Initialize Master Arrays
#########################################
masterLinArray = []
masterLogArray = []


#########################################
# Populate the Master Arrays with the column headers and 'None' Cells
#########################################

# counter = 0
# for epsilonTemp in epsilonLinRange:
#     for mATemp in mALinRange:
#         masterLinArray.append([counter,epsilonTemp,mATemp,'None'])
#         counter += 1

counter = 0
for mATemp2 in mALogRange:
    for epsilonTemp2 in epsilonLogRange:
        masterLogArray.append([counter, ' ', mATemp2, epsilonTemp2,'None','None','None','None','None','None','None'])
        counter += 1

        
#########################################
# Create a PANDAS DataFrame from the Master Arrays
#########################################

# masterLinDataFrame = pd.DataFrame(data = masterLinArray, columns=['Counter', 'mA', 'Epsilon', 'Tau_TauCross'])
masterLogDataFrame = pd.DataFrame(data = masterLogArray, \
    columns=['Counter', 'mX', 'mA', 'Epsilon',\
             'Capture',\
             'SigmaV','SommerfeldAvg','Annihilation', 'AnnihilationSomm',\
             'Tau_TauCross', 'Tau_TauCrossSomm'])

print ('Complete')

## Write the blank dataframe to an external .csv file

TO DO: search directory for file, if found, raise flag.

In [ ]:
# LinFilename = input('LinFilename: ')
# index = LinFilename.find('.')
# LinFilenamecopy = LinFilename[:index] + '_copy' + LinFilename[index:]

LogFilename = input('LogFilename: ')
writeFile = contourPath(LogFilename)

index = LogFilename.find('.')
LogFilenamecopy = LogFilename[:index] + '_copy' + LogFilename[index:]

# masterLinDataFrame.to_csv(LinFilename, index=False)
masterLogDataFrame.to_csv(writeFile, index=False)


print ('Complete')

## Define Contour File $m_X$ value

In [ ]:
filename = input('File Name: ')
writeFile = contourPath(filename)
index = filename.find('.')
filenamecopy = filename[:index] + '_copy' + filename[index:]

dataFrame = pd.read_csv(writeFile , sep = ',')

m_XValue = float(input('m_X: '))

dataFrame.set_value(0, 'mX', m_XValue)
dataFrame.to_csv(writeFile, index=False)

print ('Complete')

------------
## Contour File Sanity Check
Sanity Check

This cell serves as a sanity check to ensure that the .csv file contains what you expect it to

Current working file:
    - 100GeVHuge.csv


In [ ]:
filename = input('Filename: ')
writeFile = contourPath(filename)
index = filename.find('.')
filenamecopy = filename[:index] + '_copy' + filename[index:]
writeFileCopy = contourPath(filenamecopy)

print (pd.read_csv(writeFile, sep = ','))
print ('Complete')

## Read In CCap file and create interpolation

Working CCapFile:
    - 100GevCCap2500.csv

In [ ]:
CCapDataFile = input('CCap File:')
readFile = CCapPath(CCapDataFile)
rawData = pd.read_csv(readFile)

xMin = min(rawData['mA'])
xMax = max(rawData['mA'])
yMin = min(rawData['Epsilon'])
yMax = max(rawData['Epsilon'])


xData = np.transpose([rawData['mA']])
yData = np.transpose([rawData['Epsilon']])
CCapData = np.transpose([rawData['Capture']])


xData1 = xData.squeeze()
yData1 = yData.squeeze()
CCapData1 = np.transpose([rawData['Capture']]).squeeze()

CCapInterp = interpolate.LinearNDInterpolator((xData1,yData1),CCapData1)

print ('Complete')

## Read-in, Calculate, and Overwrite csv

The following cell:

1). Reads in the working dataframe from the previous cell

2). Calculates Values

3). Appends the calculated values to their spot in the dataframe

4). Overwrites the now updated .csv

In [ ]:
print (filename)
writeFile = contourPath(filename)
dataIn = pd.read_csv(writeFile, sep = ',')
dataIn.to_csv(writeFileCopy, sep = ',')

#########################################
# Define loop range
#########################################
loopRange = range(0,len(dataIn['Counter']))

finishedCounter = 1
for index in loopRange:
    # Scan down 'Capture' column, start calculating at first non-calculated value
    testValue = dataIn.loc[index, 'Capture']
    if (testValue == 'None'):
        
        #########################################
        # Initialize Parameters
        #########################################
        m_X = float(dataIn.loc[0,'mX'])
        m_ATemp = float(dataIn.loc[index, 'mA'])
        epsilonTemp = float(dataIn.loc[index, 'Epsilon'])
        alpha = 1./137
                        
        
        ##########################        Calculations       #########################

        # alpha_X                       : alphaTherm(m_X,m_ATemp)
        # Capture Rate                  : CCap(m_X, m_ATemp, epsilonTemp, 1./137)
        # Tree Level Cross Section      : sigmaVtree(m_X , m_ATemp, alpha_X)
        # Thermally Averaged Sommerfeld : thermAvgSommerfeld(m_X, m_ATemp, alpha_X)
        # Annihilation (no Somm)        : CAnnCalc(m_X, m_ATemp, sigmaVtree)
        # AnnihilationSomm (Somm)       : CAnnCalcSomm(m_X, m_ATemp, sigmaVtree, thermAvgSomm)
        # Tau (no Somm)                 : tau(capture, annihilation)
        # TauSomm (Somm)                : tau(capture, annihilationSomm)
        # Tau/TauCross (no Somm)        : Tau/tauCross
        # TauSomm/TauCross (Somm)       : TauSomm/tauCross

#         tauCross = yr2s(4.5e9) #sec
        #########################################
        # index = 0 Full Calculation
        #########################################
        if (index == 0):
            alpha_X = alphaTherm(m_X, m_ATemp)
            capture = CCapInterp(m_ATemp,epsilonTemp)
#             capture = CCap(m_X, m_ATemp, epsilonTemp, 1./137) # alpha_x definied by alphaTherm()
            sigmaV = sigmaVtree(m_X , m_ATemp, alpha_X)
            sommAvg = thermAvgSommerfeld(m_X, m_ATemp, alpha_X)
            annihilation = CAnnCalc(m_X, m_ATemp, sigmaV)
            annihilationSomm = CAnnSommCalc(m_X, m_ATemp, sigmaV, sommAvg)

            Tau = tau(capture,annihilation)
            TauSomm = tau(capture,annihilationSomm)

            TauTauCross = Tau/tauCross
            TauTauCrossSomm = TauSomm/tauCross

            dataIn.set_value(index, 'Capture', capture)
            dataIn.set_value(index, 'SigmaV', sigmaV)
            dataIn.set_value(index, 'SommerfeldAvg', sommAvg)
            dataIn.set_value(index, 'Annihilation', annihilation)
            dataIn.set_value(index, 'AnnihilationSomm', annihilationSomm)
            dataIn.set_value(index, 'Tau_TauCross', TauTauCross)
            dataIn.set_value(index, 'Tau_TauCrossSomm', TauTauCrossSomm)
            
            
        #########################################
        # m_A not unique -> Minimal Calculation
        #########################################
        elif(m_ATemp == float(dataIn.loc[index-1, 'mA'])):
#           Store previous data
            sigmaVPrev = float(dataIn.loc[index-1,'SigmaV'])
            sommAvgPrev = float(dataIn.loc[index-1,'SommerfeldAvg'])
            annihilationPrev = float(dataIn.loc[index-1,'Annihilation'])
            annihilationSommPrev = float(dataIn.loc[index-1,'AnnihilationSomm'])
#           Update Current data
            capture = CCapInterp(m_ATemp,epsilonTemp)
#             capture = CCap(m_X, m_ATemp, epsilonTemp, 1./137)
            sigmaV = sigmaVPrev
            sommAvg = sommAvgPrev
            annihilation = annihilationPrev
            annihilationSomm = annihilationSommPrev
            
            Tau = tau(capture,annihilation)
            TauSomm = tau(capture,annihilationSomm)

            TauTauCross = Tau/tauCross
            TauTauCrossSomm = TauSomm/tauCross
            
            #########################################
            # Overwrite current index in data file
            #########################################
            dataIn.set_value(index, 'Capture', capture)
            dataIn.set_value(index, 'SigmaV', sigmaV)
            dataIn.set_value(index, 'SommerfeldAvg', sommAvg)
            dataIn.set_value(index, 'Annihilation', annihilation)
            dataIn.set_value(index, 'AnnihilationSomm', annihilationSomm)
            dataIn.set_value(index, 'Tau_TauCross', TauTauCross)
            dataIn.set_value(index, 'Tau_TauCrossSomm', TauTauCrossSomm)
            
        #########################################
        # m_A unique -> Full Calculations
        #########################################
        elif(m_ATemp != float(dataIn.loc[index-1,'mA'])):
            alpha_X = alphaTherm(m_X, m_ATemp)
            capture = CCapInterp(m_ATemp,epsilonTemp)
#             capture = CCap(m_X, m_ATemp, epsilonTemp, 1./137) # alpha_x definied by alphaTherm()
            sigmaV = sigmaVtree(m_X, m_ATemp, alpha_X)
            sommAvg = thermAvgSommerfeld(m_X, m_ATemp, alpha_X)
            annihilation = CAnnCalc(m_X, m_ATemp, sigmaV)
            annihilationSomm = CAnnSommCalc(m_X, m_ATemp, sigmaV, sommAvg)

            Tau = tau(capture,annihilation)
            TauSomm = tau(capture,annihilationSomm)

            TauTauCross = Tau/tauCross
            TauTauCrossSomm = TauSomm/tauCross
            
            #########################################
            # Overwrite current index in data file
            #########################################
            dataIn.set_value(index, 'Capture', capture)
            dataIn.set_value(index, 'SigmaV', sigmaV)
            dataIn.set_value(index, 'SommerfeldAvg', sommAvg)
            dataIn.set_value(index, 'Annihilation', annihilation)
            dataIn.set_value(index, 'AnnihilationSomm', annihilationSomm)
            dataIn.set_value(index, 'Tau_TauCross', TauTauCross)
            dataIn.set_value(index, 'Tau_TauCrossSomm', TauTauCrossSomm)
        
        ##########################      End Calculations     #########################

        
        #########################################
        # Overwrite csv File
        #########################################
        dataIn.to_csv(writeFile, sep=',',index=False)        
        finishedCounter+=1
#         print 'Calculations for index: {0}, (m_A, Epsilon) = ({1},{2}) recorded'.format(index, m_ATemp, epsilonTemp)
        if (index % 10 == 0):
            print ('Calculations for index: {0} recorded'.format(index))
#             print ('Calculations for index: {0}, (m_A, Epsilon) = ({1},{2}) recorded'.format(index, m_ATemp, epsilonTemp))
        
    elif (testValue != 'None'):
        finishedCounter += 1
        continue
    
    if (finishedCounter == len(dataIn['Counter'])):
        print ('All Calculations Complete')
        break
        
        
dataIn.to_csv(writeFileCopy, sep=',',index=False)
    
print ('Complete')

### Manual Overwrite just for good measure

In [ ]:
print (filename)
writeFile = contourPath(filename)
writeFileCopy = contourPath(filenamecopy)
dataIn.to_csv(writeFile, sep=',',index=False)
dataIn.to_csv(writeFileCopy, sep=',',index=False)

print ('Complete')